In [ ]:
!pip install --upgrade opencv-python

In [ ]:
%%capture
!pip install netcdf4 colorcet weightedstats
import os, fnmatch, shutil, cv2, numpy as np, pandas as pd, xarray as xr, scipy.stats as st
import pylab as plt, plotly.express as px, seaborn as sns, colorcet as cc, altair as alt
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
%load_ext google.colab.data_table 
import weightedstats as ws

#!pip install --upgrade zarr gcsfs cftime nc-time-axis
#import fsspec, zarr, gcsfs; gcs = gcsfs.GCSFileSystem( token = 'anon' )

!apt-get -qq install libgeos-dev
!pip install -qq https://github.com/matplotlib/basemap/archive/master.zip
from mpl_toolkits.basemap import Basemap, maskoceans, shiftgrid

def rgb2hex( rgbstring ): 
    s = rgbstring.split( '(' )[1].split( ',' )
    return "#{0:02x}{1:02x}{2:02x}".format(max(0,min(np.int(s[0]),255)),max(0,min(np.int(s[1]),255)),max(0,min(np.int(s[2].split(')')[0]),255)))

def hex2rgb( value ):
    value = value.lstrip( '#' )
    return tuple(int(value[i:i + len(value) // 3], 16) for i in range(0, len(value), len(value) // 3))

from sklearn.neighbors import KernelDensity
def skkde( x, x_grid, bandwidth = 0.2, **kwargs ):
    kde_skl = KernelDensity( bandwidth = bandwidth, **kwargs )
    kde_skl.fit( x[:, np.newaxis] )
    log_pdf = kde_skl.score_samples( x_grid[:, np.newaxis] )
    return np.exp( log_pdf )

from google.colab import drive; drive.mount( '/content/drive' )

In [ ]:
!pip freeze

# ERA reshape and calc days over thresh and Heat load

In [ ]:
### no need to run , only run one time ###
root = '/content/drive/My Drive/data/livestock/ERA5/mean/'
filelist = [root + i for i in os.listdir(root)]
thi = xr.open_mfdataset(filelist, combine='by_coords')
thi = thi.rename({'time0':'time'})
thi = thi.groupby('time.dayofyear').median('time')
thi.to_netcdf('/content/drive/My Drive/data/livestock/ERA5/era_doy_media_new.nc4')

In [ ]:
cows = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
cows[ cows < 1 ] = 0
cows = xr.DataArray( cows, dims = {'lat': np.linspace( 90, -90, cows.shape[0] ), 'lon':  np.linspace( -180, 180, cows.shape[1] ) } )
cows = cows.coarsen( lat = 3, lon = 3, boundary = 'trim' ).sum().compute()
cows = cows.values
newmask = np.zeros_like( cows )
newmask[:,:720] = cows[:,720:]
newmask[:,720:] = cows[:,:720]

In [ ]:
ds = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = ds.lon.values
lat = ds.lat.values
test = np.ma.masked_where( ds.values[:,220:500,:] < 78.5, ds.values[:,220:500,:] ) - 78.5
test = test.mean(axis=0)
test.shape

In [ ]:
plt.pcolormesh( test )

In [ ]:
np.average(test, weights = newmask[220:500,:])

In [ ]:
np.ma.masked_where(test <30, newmask[220:500,:]).sum() / newmask.sum()

In [ ]:
ds = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = ds.lon.values
lat = ds.lat.values
ds = ds.mean(dim='dayofyear')
ds = ds.values * 0.675 + 28.56

ds, lons = shiftgrid(180., ds, lon, start=False)
lon2d, lat2d = np.meshgrid( lons, lat )
ds = maskoceans( lon2d, lat2d, ds )

colors = [rgb2hex(i) for i in px.colors.sequential.haline]
levels = np.linspace( 25, 80, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ds, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('THI Threshold', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_spatially_explicit_threshold.png', dpi=600, bbox_inches='tight', pad_inches=0)

# Load in data processed above and in the CMIP6 processing notebook 

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media_new.nc4')
lon = era.lon.values
lat = era.lat.values
doy = era.dayofyear.values
pro_thresh = 79 #69     # production threshold

era_thresh = np.ma.masked_where( era< pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )
era_pro_HL = np.sum( np.ma.masked_where( era < pro_thresh, era ) - pro_thresh, axis = 0 )
era_pro, lons = shiftgrid(180., era_pro, lon, start=False)
era_pro_HL, lons = shiftgrid(180., era_pro_HL, lon, start=False)

lon2d, lat2d = np.meshgrid( lons, lat )

era_pro = maskoceans( lon2d, lat2d, era_pro )
era_pro_HL = maskoceans( lon2d, lat2d, era_pro_HL )

In [ ]:
plt.pcolormesh(era_pro)
plt.colorbar()

In [ ]:
root = '/content/drive/My Drive/data/livestock/CMIP6/'
filelist = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp*change*.nc4')]

for file in filelist:
    thiout = np.ma.zeros( ( 366, lat.shape[0], lon.shape[0] ) )
    out    = np.ma.zeros( ( 4, lat.shape[0], lon.shape[0] ) )
    model  = file.split( '/' )[-1].split( '_chang' )[0]
    print(model)
    thi = xr.open_dataarray( file )
    days = thi.dayofyear.values
    thi = thi.values

    if days.shape[0] == 366:
        for i in range(366): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)

    elif days.shape[0] == 365:
        for i in range(60): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)
        thiout[60,:,:] = np.flip(cv2.resize(thi[59,:,:], (1440, 721) ), axis = 0)
        for i in range(61,365): thiout[i,:,:] = np.flip(cv2.resize(thi[i-1,:,:], (1440, 721) ), axis = 0)

    elif days.shape[0] == 360:
        for i in range(0,30): thiout[i,:,:] = np.flip(cv2.resize(thi[i,:,:], (1440, 721) ), axis = 0)
        thiout[30,:,:] = np.flip(cv2.resize(thi[29,:,:], (1440, 721) ), axis = 0)
        for i in range(31,153): thiout[i,:,:] = np.flip(cv2.resize(thi[i-1,:,:], (1440, 721) ), axis = 0)
        thiout[153,:,:] = np.flip(cv2.resize(thi[152,:,:], (1440, 721) ), axis = 0)
        for i in range(154,214): thiout[i,:,:] = np.flip(cv2.resize(thi[i-3,:,:], (1440, 721) ), axis = 0)
        thiout[214,:,:] = np.flip(cv2.resize(thi[210,:,:], (1440, 721) ), axis = 0)
        for i in range(215,245): thiout[i,:,:] = np.flip(cv2.resize(thi[i-4,:,:], (1440, 721) ), axis = 0)
        thiout[245,:,:] = np.flip(cv2.resize(thi[240,:,:], (1440, 721) ), axis = 0)
        for i in range(246,306): thiout[i,:,:] = np.flip(cv2.resize(thi[i-5,:,:], (1440, 721) ), axis = 0)
        thiout[306,:,:] = np.flip(cv2.resize(thi[300,:,:], (1440, 721) ), axis = 0)    
        for i in range(306,365): thiout[i,:,:] = np.flip(cv2.resize(thi[i-6,:,:], (1440, 721) ), axis = 0)
        thiout[365,:,:] = np.flip(cv2.resize(thi[359,:,:], (1440, 721) ), axis = 0) 
    
    del thi    
    thiout = thiout + era

    #new_break = thiout.mean(axis=0) * 0.346 + 49.13
    #new_break[new_break<45] = 45
    #new_break[new_break>80] = 80

    ds = xr.Dataset( coords={'lat': lat, 'lon':  lon} )
    
    temp = np.ma.masked_where( thiout < pro_thresh, thiout ).count( axis = 0 )
    ds['pro_days'] = (['lat', 'lon'],  temp)
    temp = np.sum( np.ma.masked_where( thiout < pro_thresh, thiout ) - pro_thresh, axis = 0 )
    ds['pro_HL'] = (['lat', 'lon'],  temp)
    #ds['threshold'] = (['lat', 'lon'],  new_break)

    ds.to_netcdf(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_79thresh_new.nc4')

In [ ]:
root = '/content/drive/My Drive/data/livestock/CMIP6/'
ssp126 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp126*79thresh_new.nc4')]
ssp585 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp585*79thresh_new.nc4')]

ssp5 = np.ma.zeros((12,721,1440))
for i, file in enumerate(ssp585):
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp5[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1 = np.ma.zeros((12,721,1440))
for i, file in enumerate(ssp126):
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp1[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1m = np.median( ssp1, axis = 0 )
ssp5m = np.median( ssp5, axis = 0 )

ssp1m = maskoceans( lon2d, lat2d, ssp1m )
ssp5m = maskoceans( lon2d, lat2d, ssp5m )

ssp5 = np.ma.zeros((12,721,1440))
for i, file in enumerate(ssp585):
    temp = xr.open_dataset(file)
    temp = temp.pro_HL
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp5[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1 = np.ma.zeros((12,721,1440))
for i, file in enumerate(ssp126):
    temp = xr.open_dataset(file)
    temp = temp.pro_HL
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp1[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1mh = np.median( ssp1, axis = 0 )
ssp5mh = np.median( ssp5, axis = 0 )

ssp1mh = maskoceans( lon2d, lat2d, ssp1mh )
ssp5mh = maskoceans( lon2d, lat2d, ssp5mh )

In [ ]:
np.average(ssp1m[220:500,:], weights = newmask[220:500,:])

In [ ]:
np.ma.masked_where(test <30, newmask).sum() / newmask.sum()

In [ ]:
plt.pcolormesh(ssp5mh/ssp5m)
plt.colorbar()

In [ ]:
np.ma.masked_where(ssp5m[1:,:] < 30, newmask).sum() / newmask.sum()

In [ ]:
np.ma.average(np.ma.masked_where(test > 364, ssp5m[1:,:]), weights=newmask) - np.ma.average(np.ma.masked_where(test > 364, test), weights=newmask) 

In [ ]:
np.ma.masked_where(test > 364, ssp1m[1:,:]) - np.ma.masked_where(test > 364, test)

In [ ]:
np.average(ssp1m[1:,:], weights=newmask) - np.ma.average(test, weights=newmask) 

In [ ]:
import math

In [ ]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.
    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.ma.average(values, weights=weights)
    # Fast and numerically precise:
    variance = np.ma.average((values-average)**2, weights=weights)
    return average, math.sqrt(variance)

weighted_avg_and_std(np.ma.masked_where(test > 364, ssp1m[1:,:]) - np.ma.masked_where(test > 364, test), newmask)

# Make figures

# baseline days

In [ ]:
np.arange(0,120,10)

In [ ]:
np.arange(0,360,30)

In [ ]:
plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

colors = [rgb2hex(i) for i in px.colors.sequential.solar]
levels = np.arange(0,360,30)
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Historical days over threshold', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_days_79.png', dpi=600, bbox_inches='tight', pad_inches=0)

# increase in days over threshold

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.thermal]
colors = cc.fire[0::22]
levels = [  -0.1,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110.]
levels = [  -0.1,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110]
#levels = [  -0.1,  20,  40,  60,  80,  100,  120,  140,  160,  180, 200, 220]
levels = np.array(levels) * 3
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp1m - era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.8)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Increase in days over threshold', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_days_126_79.png', dpi=1000, bbox_inches='tight', pad_inches=0)

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp5m - era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.8)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Increase in days over threshold', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_days_585_79.png', dpi=1000, bbox_inches='tight', pad_inches=0)

In [ ]:
plt.pcolormesh(ssp5m)
plt.colorbar()

# heat load all 3 panels

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.haline]
levels = np.linspace( 0, 11, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, era_pro_HL/era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Historical heat load (THIunitdays)', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_heat_load.png', dpi=600, bbox_inches='tight', pad_inches=0)

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp1mh/ssp1m , levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Increase in heat load (THIunitdays)', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_heat_load_126.png', dpi=600, bbox_inches='tight', pad_inches=0)

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp5mh/ssp5m, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Mean THI units over threshold', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_heat_load_585.png', dpi=600, bbox_inches='tight', pad_inches=0)

# Zonal means

In [ ]:
from scipy.interpolate import make_interp_spline

In [ ]:
models = ['KACE-1-0-G','BCC-CSM2-MR','UKESM1-0-LL','CNRM-CM6-1','GFDL-ESM4','CanESM5','CNRM-ESM2-1',
          'EC-Earth3-Veg','CNRM-CM6-1-HR','MRI-ESM2-0','CESM2-WACCM','MPI-ESM1-2-HR']
newlat = np.linspace(-90,90,400)
tas = np.zeros( (400, len(models)) )
thi = np.zeros( (400, len(models)) )

for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=2)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    tas[:,i] = spl1(newlat)
    thi[:,i] = spl2(newlat)

tasmin = np.min(tas, axis = 1)
tasmax = np.max(tas, axis = 1)
tasmed = np.mean(tas, axis = 1)

thimin = np.min(thi, axis = 1)
thimax = np.max(thi, axis = 1)
thimed = np.mean(thi, axis = 1)

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (1, 2 ) )

newlat = np.linspace( -90, 90, 400 )

color = '#bf9000' # '#4a86e8'

for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp126_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = color, lw = 0.4, alpha = 0.8, zorder = 1)

plt.fill_betweenx(newlat, tasmin, tasmax, facecolor = 'black', alpha = 0.15, zorder = 2)
plt.fill_betweenx(newlat, thimin, thimax, facecolor = color, alpha = 0.2, zorder = 3)

plt.plot(tasmed, newlat, color = 'black', lw = 1.5, alpha = 1, zorder = 4)
plt.plot(thimed, newlat, color = color, lw = 2, alpha = 1, zorder = 5)

plt.xlim( 0, 15 )
plt.xticks( [0,5,10,15], [0,5,10,15] )
plt.ylim( -56, 76 )
sns.despine( offset = 3, trim = True )
plt.yticks( rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/ssp126_zonal_mean.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
models = ['KACE-1-0-G','BCC-CSM2-MR','UKESM1-0-LL','CNRM-CM6-1','GFDL-ESM4','CanESM5','CNRM-ESM2-1',
          'EC-Earth3-Veg','CNRM-CM6-1-HR','MRI-ESM2-0','CESM2-WACCM','MPI-ESM1-2-HR']
newlat = np.linspace(-90,90,400)
tas = np.zeros( (400, len(models)) )
thi = np.zeros( (400, len(models)) )

red = '#980000'

for i, model in enumerate(models):
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=2)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=2)
    tas[:,i] = spl1(newlat)
    thi[:,i] = spl2(newlat)

tasmin = np.min(tas, axis = 1)
tasmax = np.max(tas, axis = 1)
tasmed = np.mean(tas, axis = 1)

thimin = np.min(thi, axis = 1)
thimax = np.max(thi, axis = 1)
thimed = np.mean(thi, axis = 1)

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (1, 2 ) )

newlat = np.linspace( -90, 90, 400 )

color = '#bf9000' # '#4a86e8'

for model in models:
    x = np.genfromtxt(f'/content/drive/My Drive/data/livestock/CMIP6/{model}_ssp585_zonal_mean.csv', delimiter=',')
    spl1 = make_interp_spline(x[:,0], x[:,1], k=1)
    spl2 = make_interp_spline(x[:,0], x[:,2], k=1)
    plt.plot(spl1(newlat), newlat, color = 'black', lw = 0.4, alpha = 0.4, zorder = 0)
    plt.plot(spl2(newlat), newlat, color = color, lw = 0.4, alpha = 0.8, zorder = 1)

plt.fill_betweenx(newlat, tasmin, tasmax, facecolor = 'black', alpha = 0.15, zorder = 2)
plt.fill_betweenx(newlat, thimin, thimax, facecolor = color, alpha = 0.2, zorder = 3)

plt.plot(tasmed, newlat, color = 'black', lw = 1.5, alpha = 1, zorder = 4)
plt.plot(thimed, newlat, color = color, lw = 2, alpha = 1, zorder = 5)

plt.xlim(0,15)
plt.xticks([0,5,10,15],[0,5,10,15])
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.ylim(-56,76)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/ssp585_zonal_mean.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

## just africa, for IPCC

In [ ]:
colors = cc.fire[0::18]
levels = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7]
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 10, 10 ) )

m = Basemap( projection ='cyl', llcrnrlon = -17.9, llcrnrlat = -35, urcrnrlon = 51.7, urcrnrlat = 37.7, resolution='i' )
x, y = m( lon2d, lat2d )

m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp1mh/ssp1m - era_pro_HL/era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.8)

CB = plt.colorbar( orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Increase in heat load (THIunitdays)', fontsize=8)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_heat_load_126_IPCC.png', dpi=600, bbox_inches='tight', pad_inches=0)

plt.close('all')
fig = plt.figure( figsize = ( 10, 10 ) )

m = Basemap( projection ='cyl', llcrnrlon = -17.9, llcrnrlat = -35, urcrnrlon = 51.7, urcrnrlat = 37.7, resolution='i' )
x, y = m( lon2d, lat2d )

m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp5mh/ssp5m - era_pro_HL/era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.8)

CB = plt.colorbar( orientation = "horizontal", ticks = levels)
CB.ax.set_xticklabels([0, ' ', 1, ' ', 2, ' ', 3, ' ', 4, ' ', 5, ' ', 6, ' ', 7])
CB.ax.tick_params( labelsize = 10 )

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_heat_load_585_IPCC.png', dpi=600, bbox_inches='tight', pad_inches=0)

# Bivariate

In [ ]:
#Bivaraite colormaps
colors = ['#e8e8e8', '#b0d5df', '#64acbe', '#e4acac', '#ad9ea5', '#627f8c', '#c85a5a', '#985356', '#574949'] #red blue bivariate
#colors = ['#e8e8e8', '#dfb0d6', '#be64ac', '#ace4e4', '#a5add3', '#8c62aa', '#5ac8c8', '#5698b9', '#574949'] #pink blue bivariate
levels = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
cmap, norm = flc( levels = levels, colors = colors )

### to make bivarate colorbar
fig = plt.figure( figsize = ( 4, 4 ) )
x = np.array([[5,11,17],[3,9,15],[1,7,13]])
plt.pcolor(np.flip(x, axis =0), cmap = cmap)
plt.axis('off')
plt.savefig('/content/drive/My Drive/figures/livestock/rd_bu_bivariate.png', dpi = 600, bbox_inches='tight', pad_inches=0)

### text numbers

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media_new.nc4')
late  = era.lat.values
lone = era.lon.values
pro_thresh = 79    # production threshold
era_thresh = np.ma.masked_where( era < pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )
era_pro_HL = np.sum( np.ma.masked_where( era < pro_thresh, era ) - pro_thresh, axis = 0 )
era_pro, lons = shiftgrid( 180., era_pro, lone, start = False )

### Cattle Density data from FAO ###
img = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
#area = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/8_Areakm.tif', cv2.IMREAD_UNCHANGED)
#img = img / area
img[img<1] = 0

In [ ]:
plt.pcolormesh(era_pro)

In [ ]:
plt.pcolormesh(img)
plt.colorbar()

In [ ]:
lon = np.arange(0,360,4320)
lat = np.linspace(90,-90,2160)
lat

In [ ]:
def weighted_avg_std(values, weights):
    #Return the weighted average and standard deviation.
    #values, weights -- Numpy ndarrays with the same shape.
    average = np.ma.average(values, weights=weights)
    variance = np.ma.average( (values-average)**2, weights=weights )
    return ( average, np.sqrt( variance ) )

In [ ]:
v1 = era_pro.astype(float)
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
weighted_avg_std(v1, weights=img)

In [ ]:
plt.pcolormesh(era_pro, vmin = 0, vmax = 365)

In [ ]:
v1 = ssp5m - ssp1m
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
era_r = cv2.resize( era_pro.astype(float), (img.shape[1], img.shape[0]) )
v1 = np.ma.masked_where(era_r > 356 , v1)
weighted_avg_std(v1, weights=img)

In [ ]:
plt.pcolormesh(v1, vmin = 0, vmax = 365) 

In [ ]:
plt.pcolormesh( ssp5m )
plt.colorbar()

In [ ]:
plt.pcolormesh( ssp1m )
plt.colorbar()

In [ ]:
v1 = era_pro.astype(float)
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
weighted_avg_std(v1, img)

In [ ]:
v1 = ssp1m
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
np.ma.average(v1, weights=img)

In [ ]:
v1 = ssp1mh
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
v1 = np.nan_to_num(v1)
weighted_avg_std(v1/203, weights=img)

In [ ]:
v1 = ssp5m
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
v1 = np.nan_to_num(v1)
np.ma.average(v1, weights=img)

In [ ]:
242 - 187

In [ ]:

img.shape[0]

In [ ]:
img.shape[1] 

In [ ]:
v1 = ssp5m - era_pro
v1 = cv2.resize( v1, (img.shape[1], img.shape[0]) )
v1 = np.nan_to_num(v1)
weighted_avg_std(v1, img)

In [ ]:
plt.pcolormesh(v1)
plt.colorbar()

In [ ]:
from scipy.interpolate import interp2d

In [ ]:
lont.shape

In [ ]:
### Cattle Density data from FAO ###
img = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/6_Ct_2010_Aw.tif', cv2.IMREAD_UNCHANGED)
area = cv2.imread('/content/drive/My Drive/data/livestock/FAO_distribution/8_Areakm.tif', cv2.IMREAD_UNCHANGED)
img = img / area

newlat = np.linspace(90,-90,img.shape[0])
newlon = np.linspace(-180,180,img.shape[1])

v1 = era_pro
#v1, lont = shiftgrid( 180., v1, lone, start = False )
f = interp2d(lons, late, v1, kind='linear')
v1 = f(newlon, newlat)
v1 = np.flip(v1, axis = 0)
low = 1
high = 50

new = np.zeros_like(v1)
new[ ( img < 1 )  & ( v1 < low ) ] = 1
new[ ( img < 10 ) & ( img > 1 ) & ( v1 < low ) ] = 3
new[ ( img > 10 ) & ( v1 < low ) ] = 5

new[ ( img < 1 )  & ( v1 < high ) & ( v1 > low ) ] = 7
new[ ( img < 10 ) & ( img > 1 ) & ( v1 < high ) & ( v1 > low ) ] = 9
new[ ( img > 10 ) & ( v1 < high ) & ( v1 > low ) ] = 11

new[ ( img < 1 )  & ( v1 > high ) ] = 13
new[ ( img < 10 ) & ( img > 1 ) & ( v1 > high ) ] = 15
new[ ( img > 10 ) & ( v1 > high ) ] = 17

lat = np.linspace( 90, -90, new.shape[0] )
lon = np.linspace( -180, 180, new.shape[1] )
lon2d, lat2d = np.meshgrid( lon, lat )
new = maskoceans( lon2d, lat2d, new )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

m = Basemap( projection = 'robin', lon_0 = 0, resolution = 'l' )
x, y = m( lon2d, lat2d )

m.drawmapboundary( fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents( color = '#bdbdbd', alpha = 0.3)
m.contourf( x, y, new, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

plt.savefig('/content/drive/My Drive/figures/livestock/bivaraite.png', dpi = 600, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
plt.pcolormesh(new)
plt.colorbar()

In [ ]:
plt.pcolormesh(v1)

In [ ]:
lat = np.linspace( 90, -90, my_arr.shape[0] )
lon = np.linspace( -180, 180, my_arr.shape[1] )
lon2d, lat2d = np.meshgrid( lon, lat )
my_arr = maskoceans( lon2d, lat2d, my_arr )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4  ) )

m = Basemap( projection = 'robin', lon_0 = 0, resolution = 'l' )
x, y = m( lon2d, lat2d )

m.drawmapboundary( fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents( color = '#bdbdbd', alpha = 0.3)
m.contourf( x, y, my_arr, levels = levels, extend = 'max', cmap = cmap, zorder=10, alpha = 0.9)

plt.savefig('...path/my_bivaraite.png', dpi = 600, bbox_inches = 'tight', pad_inches = 0 )

### supplemental contour

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = era.lon.values
lat = era.lat.values
doy = era.dayofyear.values
pro_thresh = 69     # production threshold

era_thresh = np.ma.masked_where( era< pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )
era_pro, lons = shiftgrid(180., era_pro, lon, start=False)

lon2d, lat2d = np.meshgrid( lons, lat )

era_pro = maskoceans( lon2d, lat2d, era_pro )

In [ ]:
root = '/content/drive/My Drive/data/livestock/CMIP6/'
ssp126 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp126*69thresh.nc4')]
ssp585 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp585*69thresh.nc4')]

plt.close('all')
fig = plt.figure( figsize = ( 10, 4  ) )

gs = GridSpec(1,1 )
gs.update(wspace = 0, hspace = 0.01)

lon2d, lat2d = np.meshgrid(lons, lat)
m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)


era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contour(x, y, era_pro, levels = [60], colors = ['black'], linewidths = [0.5], zorder=10)
m.contourf(x, y, era_pro, levels = [60,370], colors = ['black'], zorder=1, alpha = 0.3)

for file in ssp126:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [60], colors = ['blue'], linewidths = [0.5], zorder=10, alpha=0.3)

for file in ssp585:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [60], colors = ['red'], linewidths = [0.5], zorder=10, alpha=0.3)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_pro_doy_countour_60.png', dpi=600, bbox_inches='tight', pad_inches=0)

# supplemental regions

In [ ]:
### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
regions = regions.replace({'United States':'United States of America',
                           'Bolivia': 'Bolivia (Plurinational State of)',
                           'Libyan Arab Jamahiriya': 'Libya',
                           'Venezuela':'Venezuela (Bolivarian Republic of)',
                           'Russia':'Russian Federation',
                           'Cote dIvoire': "Côte d'Ivoire",
                           'Laos': "Lao People's Democratic Republic",
                           'Burma':'Myanmar',
                           'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                           'Korea,  Republic of': 'Republic of Korea'})

### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
dfc = dfc[dfc.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )

smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

#dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
dfc = dfc.drop( columns = ['countryname','Item'] )

df = pd.merge(regions, dfc, on = 'ISO')

cmg  = xr.open_dataarray( '/content/drive/My Drive/data/area_masks/gadm36_countries_FOA_animal_dist_resolution.nc4' )
lat = cmg.lat.values
lon = cmg.lon.values
cmg = cmg.values

out = np.zeros_like( cmg )
for i in df.maskid.unique():
    dft = df[df.maskid == i ]
    if dft.R5_region.unique()  == 'R5.2OECD': out[cmg == i] = 1
    elif dft.R5_region.unique()  == 'R5.2ASIA': out[cmg == i] = 2
    elif dft.R5_region.unique()  == 'R5.2LAM':  out[cmg == i] = 3
    elif dft.R5_region.unique()  == 'R5.2MAF':  out[cmg == i] = 4
    elif dft.R5_region.unique()  == 'R5.2REF':  out[cmg == i] = 5

out = np.ma.masked_where(out == 0, out)

colors = [rgb2hex(i) for i in px.colors.sequential.deep[2::3]] + ['#969696']
levels = [0,1.5,2.5,3.5,4.5,5.6]
cmap, norm = flc( levels = levels, colors = colors)

lon2d, lat2d = np.meshgrid( lon, lat )

plt.close('all')
fig = plt.figure( figsize = ( 7, 3.5  ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.5, 1, 0.5] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'i', area_thresh = 1000 )
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, out, levels = levels, extend = 'min', cmap = cmap, zorder=1, alpha = 0.7)
m.drawcountries( color = 'black', linewidth = 0.2 , zorder = 2)
m.drawcoastlines( color = 'black', linewidth = 0.2 , zorder = 2)

plt.savefig('/content/drive/My Drive/figures/livestock/region_map_supplement.png', dpi=600, bbox_inches='tight', pad_inches=0)

### OLD ##

In [ ]:
### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
regions = regions.replace({'United States':'United States of America',
                           'Bolivia': 'Bolivia (Plurinational State of)',
                           'Libyan Arab Jamahiriya': 'Libya',
                           'Venezuela':'Venezuela (Bolivarian Republic of)',
                           'Russia':'Russian Federation',
                           'Cote dIvoire': "Côte d'Ivoire",
                           'Laos': "Lao People's Democratic Republic",
                           'Burma':'Myanmar',
                           'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                           'Korea,  Republic of': 'Republic of Korea'})

### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
dfc = dfc[dfc.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )

smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

#dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
dfc = dfc.drop( columns = ['countryname','Item'] )

df = pd.merge(regions, dfc, on = 'ISO')

cmg  = xr.open_dataarray( '/content/drive/MyDrive/data/area_masks/gadm36_countries_0.25_degrees.nc4' )
lat = cmg.lat.values
lon = cmg.lon.values
cmg = cmg.values

out = np.zeros_like( cmg )
for i in df.maskid.unique():
    dft = df[df.maskid == i ]
    if dft.R5_region.unique()  == 'R5.2OECD': out[cmg == i] = 1
    elif dft.R5_region.unique()  == 'R5.2ASIA': out[cmg == i] = 2
    elif dft.R5_region.unique()  == 'R5.2LAM':  out[cmg == i] = 3
    elif dft.R5_region.unique()  == 'R5.2MAF':  out[cmg == i] = 4
    elif dft.R5_region.unique()  == 'R5.2REF':  out[cmg == i] = 5

ds = xr.DataArray(out, dims={'lat':lat,'lon':lon})
ds.to_netcdf('/content/drive/Shareddrives/ClimateAg/Haynes/SSP_demand_changes/ssp_regions_era5_res.nc4')

In [ ]:
plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'#980000','ytick.color':'black','xtick.color':'#980000','axes.facecolor':'white', 'hatch.color': 'gray'} )

plt.close('all')
fig = plt.figure( figsize = (2.5, 3 ) )

newlat = np.linspace(-90,90,400)

color = '#4a86e8' 
color = '#980000'

plt.plot( era_pro.mean( axis = 1 ), lat, color = color, lw = 0.8, alpha = 1, zorder = 4)
plt.plot( era_pro_HL.mean( axis = 1 )/365*20, lat, color = '#4a86e8' , lw = 0.8, alpha = 1, zorder = 5)

plt.xlim(0,380)
plt.xticks( [0,180,360], [0,180,360] )
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.axhline(0, lw = 0.3, linestyle = '--', color = 'black', zorder = 0)
plt.ylim(-56,76)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/fig3_zonal_mean_1.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'#980000','ytick.color':'black','xtick.color':'#980000','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (2.5, 3 ) )

newlat = np.linspace(-90,90,400)

color = '#4a86e8' 
color = '#980000'

plt.plot( era_pro.mean( axis = 1 ), lat, color = 'black', lw = 0.8, alpha = 1, zorder = 4)
plt.plot( era_pro_HL.mean( axis = 1 )/365*20, lat, color = '#4a86e8' , lw = 0.8, alpha = 1, zorder = 5)

plt.xlim(0,10)
plt.xticks( [0,180,360], [0,180,360] )
#plt.xlabel('Temperature increase ($\degree$C | THI over threshold)')
#plt.ylabel('Latitude')
plt.axhline(0, lw = 0.3, linestyle = '--', color = 'black', zorder = 0)
plt.ylim(-56,76)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/fig3_zonal_mean_1.png', dpi = 800, bbox_inches = 'tight', pad_inches = 0 )

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.speed_r]
levels = np.linspace( 0, 363, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 10, 3  ) )

gs = GridSpec(2, 4, height_ratios = [1, 0.05], width_ratios = [1, 1, 1,1] )
gs.update(wspace = 0, hspace = 0.01)
lon2d, lat2d = np.meshgrid( lons, lat )

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0:2])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contourf(x, y, ssp1m, levels = levels, extend = 'max', cmap = cmap, zorder=10)
plt.title('ssp126')

ax1 = plt.subplot(gs[0,2:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp5m, levels = levels, extend = 'max', cmap = cmap, zorder=10)
plt.title('ssp585')

ax2 = plt.subplot( gs[1,1:3] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Days per year over threshold', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
np.linspace( 1, 56, 12 )

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.deep]
levels = np.linspace( 1, 56, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = (10, 6 ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)
lon2d, lat2d = np.meshgrid( lons, lat )

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contourf(x, y, ssp5m - ssp1m, levels = levels, extend = 'max', cmap = cmap, zorder=10)
plt.title('mitigation (ssp585 - ssp126)')

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Days per year mitigated', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
np.linspace( 0, 4, 12 )

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.haline]
levels = np.linspace( 0, 4, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = (10, 6 ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.2, 1, 0.2] )
gs.update(wspace = 0, hspace = 0.01)
lon2d, lat2d = np.meshgrid( lons, lat )

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contourf(x, y, (ssp5m - ssp1m) / 365, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax2 = plt.subplot( gs[1,1:2] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Heat load mitigated (THIunitdays/day)', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.thermal]
levels = np.linspace( 1, 133, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 10, 3  ) )

gs = GridSpec(2, 4, height_ratios = [1, 0.05], width_ratios = [1, 1, 1,1] )
gs.update(wspace = 0, hspace = 0.01)
lon2d, lat2d = np.meshgrid( lons, lat )

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0:2])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contourf(x, y, ssp1m - era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10)
plt.title('ssp126')

ax1 = plt.subplot(gs[0,2:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, ssp5m - era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10)
plt.title('ssp585')

ax2 = plt.subplot( gs[1,1:3] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Increase in days per year over threshold', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
plt.close('all')
fig = plt.figure( figsize = ( 10, 4  ) )

gs = GridSpec(1,1 )
gs.update(wspace = 0, hspace = 0.01)

lon2d, lat2d = np.meshgrid(lons, lat)
m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contour(x, y, era_pro, levels = [360], colors = ['black'], linewidths = [0.5], zorder=10)
m.contourf(x, y, era_pro, levels = [0,360], colors = ['black'], zorder=1, alpha = 0.3)

for file in ssp126:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [360], colors = ['blue'], linewidths = [0.5], zorder=10, alpha=0.3)

for file in ssp585:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [360], colors = ['red'], linewidths = [0.5], zorder=10, alpha=0.3)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_pro_doy_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.thermal]
levels = np.linspace( 0, 330, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 10, 4  ) )

gs = GridSpec(1,2 )
gs.update(wspace = 0, hspace = 0.01)

lon2d, lat2d = np.meshgrid(lons, lat)
m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
era_pro = maskoceans( lon2d, lat2d, era_pro )
m.contour(x, y, era_pro, levels = [360], colors = ['black'],  linewidths = [0.5], zorder=10)

for file in ssp126:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [360], colors = ['blue'], linewidths = [0.5], zorder=10,alpha=0.3)

ax1 = plt.subplot(gs[0,1])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contour(x, y, era_pro, levels = [360], colors = ['black'], linewidths = [0.5], zorder=10,alpha=0.3)
for file in ssp585:
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    temp = maskoceans( lon2d, lat2d, temp )
    m.contour(x, y, temp, levels = [360], colors = ['red'], linewidths = [0.5], zorder=10)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.matter]
levels = np.linspace( 0, 330, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 8, 4 ) )

gs = GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.25, 1, 0.15] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )
v1 = maskoceans( lon2d, lat2d, v1 )

ax1 = plt.subplot(gs[0,0:3])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, v1, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax2 = plt.subplot( gs[1,1] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Days per year over threshold (1984:2014)', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_hist.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.thermal]
#colors = px.colors.sequential.Plotly3
levels = np.linspace( 0, 330, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 12, 10 ) )

gs = GridSpec(4, 4, height_ratios = [1,1,1, 0.05], width_ratios = [1, 1, 1,1] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

era_pro = maskoceans( lon2d, lat2d, era_pro )
era_dea = maskoceans( lon2d, lat2d, era_dea )
ssp585p = maskoceans( lon2d, lat2d, ssp585p )
ssp585d = maskoceans( lon2d, lat2d, ssp585d )
ssp245p = maskoceans( lon2d, lat2d, ssp245p )
ssp245d = maskoceans( lon2d, lat2d, ssp245d )

ax1 = plt.subplot(gs[0,0:2])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, era_pro, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax1 = plt.subplot(gs[1,0:2])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, ssp245p, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax1 = plt.subplot(gs[2,0:2])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, ssp585p, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax1 = plt.subplot(gs[0,2:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, era_dea, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax1 = plt.subplot(gs[1,2:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, ssp245d, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax1 = plt.subplot(gs[2,2:])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.contourf(x, y, ssp585d, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax2 = plt.subplot( gs[3,1:3] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Days per year over threshold', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_6pan_thermal.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.thermal]
#colors = px.colors.sequential.Plotly3
levels = np.linspace( 0, 330, 12 )
cmap, norm = flc( levels = levels, colors = colors, extend='max' )

plt.close('all')
fig = plt.figure( figsize = ( 10, 4  ) )

gs = GridSpec(1,2 )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'robin', lon_0 = 0, resolution = 'l')
x, y = m( lon2d, lat2d )

ax1 = plt.subplot(gs[0,0])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contour(x, y, era_pro, levels = [360], colors = ['black'],  linewidths = [0.5], zorder=10)
m.contour(x, y, ssp245p, levels = [360], colors = ['blue'],  linewidths = [0.5], zorder=10)
m.contour(x, y, ssp585p, levels = [360], colors = ['red'],  linewidths = [0.5], zorder=10)

ax1 = plt.subplot(gs[0,1])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contour(x, y, era_dea, levels = [360], colors = ['black'], linewidths = [0.5], zorder=10)
m.contour(x, y, ssp245d, levels = [360], colors = ['blue'],  linewidths = [0.5], zorder=10)
m.contour(x, y, ssp585d, levels = [360], colors = ['red'], linewidths = [0.5], zorder=10)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_dothres_countour.png', dpi=600, bbox_inches='tight', pad_inches=0)

In [ ]:
px.colors.sequential.swatches()

In [ ]:
thi = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
thi = thi.values
pub_thresh = 68.2
thi = thi - pub_thresh
thi[thi<0] = 0
thi = np.sum(thi, axis = 0)
thi = maskoceans( lon2d, lat2d, thi )
thi = np.mean(thi, axis = 1)

In [ ]:
plt.close('all')
plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 8, 'ytick.labelsize' : 8, 'legend.fontsize': 8, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1.4, 'ytick.major.size':1.4,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':1, 'ytick.major.pad':1, 'axes.labelpad':0.2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor': 'black','ytick.color': 'black','xtick.color': 'black'})

fig = plt.figure( figsize = ( 2.5, 2.5 ) )

plt.plot( thi, lat, color = 'black', linestyle = '--')
plt.plot( thi+1000, lat, color = px.colors.sequential.Plotly3[0] )
plt.plot( thi+1500, lat, color = px.colors.sequential.Plotly3[5] )
plt.plot( thi+2000, lat, color = px.colors.sequential.Plotly3[9] )
plt.xlabel('Zonal mean heat load (THIdays)')
plt.xlim(0,10000)
sns.despine( trim = True, offset=2)
plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_hist.png', dpi=600, bbox_inches='tight', pad_inches=0)

### calculate climate model deltas

## Reshape to ERA-5, calc days-over-thresh and heat load

## Calc doy ERA THI values

In [ ]:
root = '/content/drive/My Drive/data/livestock/ERA5/thom/mean/'
filelist = [root + i for i in os.listdir(root)]
thi = xr.open_mfdataset(filelist, combine='by_coords')
thi = thi.rename({'time0':'time'})
thi = thi.groupby('time.dayofyear').median('time')
thi.to_netcdf('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
#thi = thi.where(thi>68.2).THI.values
#thi 

In [ ]:
thi = xr.open_mfdataset(filelist, combine='by_coords')
thi = thi.groupby('time.dayofyear').median('time')
thi = thi.where(thi<68.2)
thi

In [ ]:
plt.close('all')
fig = plt.figure( figsize = ( 8, 4 ) )

gs = gridspec.GridSpec(2, 3, height_ratios = [1, 0.05], width_ratios = [0.25, 1, 0.15] )
gs.update(wspace = 0, hspace = 0.01)

m = Basemap(projection = 'eck4', lon_0 = 0, resolution = 'i')

v1 = np.load('/content/drive/My Drive/data/livestock/CMIP6THIchange/GFDL-CM4_2010_THIr.npy')
v2 = v1.copy() 
n = 144 # 256 for EC-EARTH
v1[:,:n] = v2[:,n:]
v1[:,n:] = v2[:,:n]
lat = ds2.lat.values
lon = np.linspace(-180, 180, v1.shape[1])

lon2d, lat2d = np.meshgrid(lon, lat)
x, y = m(lon2d, lat2d)
v1 = maskoceans(lon2d, lat2d, v1)
colors = [rgb2hex(i) for i in px.colors.sequential.deep]
levels = [1, 20, 40, 60, 80, 100, 120, 140, 160]
cmap, norm = from_levels_and_colors(levels=levels, colors=colors[1:10], extend = 'max')

ax1 = plt.subplot(gs[0,0:3])
m.drawmapboundary(fill_color = 'white', color = 'white', linewidth = 0.0)
m.fillcontinents(color = '#bdbdbd', alpha = 0.3)
m.contourf(x, y, v1, levels = levels, extend = 'max', cmap = cmap, zorder=10)

ax2 = plt.subplot( gs[1,1] )
CB = plt.colorbar( cax = ax2, orientation = "horizontal" )
CB.ax.tick_params( labelsize = 7 )
CB.set_label('Days per year over threshold (1:2010)', fontsize=7)

plt.savefig('/content/drive/My Drive/figures/livestock/era_THI_hist.png', dpi=5600, bbox_inches='tight', pad_inches=0)